<a href="https://colab.research.google.com/github/AchmadAlvin/Absensi-face-recognition-esp32/blob/main/cam_colab_mqtt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install paho-mqtt flask opencv-python numpy requests pyngrok
!pip install paho-mqtt opencv-python numpy requests flask
!pip install pyngrok
!pip install flask opencv-python numpy pyngrok

In [ ]:
from pyngrok import ngrok

# Ganti token ngrok (jangan lupa buat akun e ya cuy)
NGROK_AUTH_TOKEN = ""

# Setel
!ngrok authtoken $NGROK_AUTH_TOKEN


In [ ]:
import cv2
import numpy as np
import base64
import paho.mqtt.client as mqtt
from flask import Flask, request
from pyngrok import ngrok

app = Flask(__name__)

# ========== Setup MQTT ==========
broker = "broker.emqx.io"
mqtt_topic = "la/server/url"

client = mqtt.Client()
client.connect(broker, 1883, 60)

NGROK_AUTH_TOKEN = ""
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

public_url = ngrok.connect(5000).public_url
print("Ngrok Public URL:", public_url)


client.publish(mqtt_topic, public_url)
print(f"URL {public_url} berhasil dikirim ke MQTT!")

# get gambar
@app.route("/", methods=["POST"])
def receive_image():
    try:
        img_data = request.data  # Terima gambar dari ESP32-CAM
        np_arr = np.frombuffer(img_data, np.uint8)
        img = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)

        # saving
        cv2.imwrite("received_image.jpg", img)
        print("Gambar diterima dan disimpan!")

        # (Nantinya, di sini akan ditambahkan Face Recognition)
        return "Gambar diterima!", 200
    except Exception as e:
        print("Error:", e)
        return "Terjadi kesalahan", 500

# Flask
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)
